In [2]:
# === Grab yesterday's Ethereum-trading Reddit posts (UTC) -> CSV ===
# 依赖：praw, pandas, python-dotenv（会自动安装）
import os, sys, time, re, datetime as dt
from datetime import timezone
from typing import List

def _pip_install(pkgs: List[str]):
    import importlib, subprocess
    for p in pkgs:
        try:
            importlib.import_module(p.split("==")[0])
        except Exception:
            subprocess.check_call([sys.executable, "-m", "pip", "install", p])

_pip_install(["praw", "pandas", "python-dotenv"])

from dotenv import load_dotenv
load_dotenv()  # 读取项目根目录的 .env；若 Notebook 在子目录，先 os.chdir 到根目录
import praw
import pandas as pd

# 读取凭证（来自 .env）
CLIENT_ID     = os.getenv("REDDIT_CLIENT_ID")
CLIENT_SECRET = os.getenv("REDDIT_CLIENT_SECRET")
USER_AGENT    = os.getenv("REDDIT_USER_AGENT", "eth-sentiment-bot/0.1")

if not CLIENT_ID or not CLIENT_SECRET or not USER_AGENT:
    raise RuntimeError("缺少 Reddit 凭证：请在 .env 中设置 REDDIT_CLIENT_ID / REDDIT_CLIENT_SECRET / REDDIT_USER_AGENT")

# 子版块与关键词（可按需增减）
SUBREDDITS = ["ethereum", "ethfinance", "ethtrader", "CryptoCurrency", "defi", "ethdev"]
KEYWORDS = [
    r"\beth\b", r"\bethereum\b", r"\beth/usdt\b", r"\bethusd\b", r"\bethusdt\b",
    r"\bspot\b", r"\bfutures?\b", r"\bperps?\b", r"\btrade|trading|trader\b",
    r"\bposition\b", r"\blong\b", r"\bshort\b", r"\bentry\b", r"\bexit\b",
    r"\bmarket\b", r"\border\b", r"\bliquidation\b", r"\bhedge\b", r"\bleverage\b"
]
KW_REGEX = re.compile("|".join(KEYWORDS), flags=re.IGNORECASE)

MAX_PER_SUBREDDIT = 5000
REQUEST_SLEEP = 0.5
SAVE_DIR = "./data/reddit/yesterday"

# 昨日（UTC）窗口
now_utc = dt.datetime.now(timezone.utc)
day_end = dt.datetime(year=now_utc.year, month=now_utc.month, day=now_utc.day, tzinfo=timezone.utc)
day_start = day_end - dt.timedelta(days=1)
D = day_start.date()
print(f"[UTC window] {day_start.isoformat()} -> {day_end.isoformat()} (D={D})")

# Reddit 客户端（只读）
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT,
    ratelimit_seconds=5,
)
reddit.read_only = True

def _to_utc(ts: float) -> dt.datetime:
    return dt.datetime.fromtimestamp(ts, tz=timezone.utc)

def _match_eth_trading(title: str, selftext: str) -> bool:
    return bool(KW_REGEX.search(f"{title or ''}\n{selftext or ''}"))

# 采集
rows = []
os.makedirs(SAVE_DIR, exist_ok=True)

for sub in SUBREDDITS:
    print(f"\n[collect] r/{sub} ...")
    count = 0
    try:
        for submission in reddit.subreddit(sub).new(limit=None):
            created = _to_utc(getattr(submission, "created_utc", 0.0))
            if created >= day_end:
                continue
            if created < day_start:
                break

            if not _match_eth_trading(submission.title, submission.selftext):
                continue

            rows.append(dict(
                id=submission.id,
                subreddit=submission.subreddit.display_name,
                created_utc=created.isoformat(),
                title=submission.title or "",
                selftext=submission.selftext or "",
                url=submission.url or "",
                is_self=bool(submission.is_self),
                author=str(submission.author) if submission.author else None,
                score=int(submission.score or 0),
                upvote_ratio=float(submission.upvote_ratio or 0.0),
                num_comments=int(submission.num_comments or 0),
                over_18=bool(getattr(submission, "over_18", False)),
                stickied=bool(getattr(submission, "stickied", False)),
                crosspost_parent=getattr(submission, "crosspost_parent", None),
                permalink=f"https://www.reddit.com{submission.permalink}" if getattr(submission, "permalink", None) else "",
                fetched_at=dt.datetime.now(timezone.utc).isoformat(),
                fetch_window_start=day_start.isoformat(),
                fetch_window_end=day_end.isoformat(),
                fetch_version="submissions_only_v1"
            ))
            count += 1
            if count % 200 == 0:
                print(f"  r/{sub}: matched {count} ...")
            if count >= MAX_PER_SUBREDDIT:
                print(f"  r/{sub}: hit MAX_PER_SUBREDDIT={MAX_PER_SUBREDDIT}, stop.")
                break
            time.sleep(REQUEST_SLEEP)
    except Exception as e:
        print(f"  [warn] subreddit {sub} failed: {e}")

# 去重并保存 CSV
df = pd.DataFrame(rows)
csv_path = os.path.join(SAVE_DIR, f"reddit_eth_submissions_{D}.csv")

if df.empty:
    print("\n[summary] No matched submissions found. 可调整关键词/子版块后重试。")
else:
    df.sort_values(["id", "score", "fetched_at"], ascending=[True, False, False], inplace=True)
    df = df.drop_duplicates(subset=["id"], keep="first").reset_index(drop=True)
    df["is_crosspost"] = df["crosspost_parent"].notna()
    df.to_csv(csv_path, index=False, encoding="utf-8")

    dt_min = pd.to_datetime(df["created_utc"]).min()
    dt_max = pd.to_datetime(df["created_utc"]).max()
    print("\n[summary]")
    print(f"  window (UTC): {day_start.isoformat()} → {day_end.isoformat()}")
    print(f"  collected rows: {len(df)}  (after de-dup)")
    print(f"  time coverage:  {dt_min} → {dt_max}")
    print(f"  subreddits:     {', '.join(sorted(df['subreddit'].unique()))}")
    print(f"  saved csv:      {csv_path}")

    # 预览前 10 条
    preview = (
        df[["created_utc","subreddit","score","num_comments","title"]]
        .sort_values(["score","num_comments"], ascending=False)
        .head(10)
    )
    print("\n[top 10 by score/num_comments]")
    for _, r in preview.iterrows():
        print(f"- [{r['created_utc']}] r/{r['subreddit']} | score={r['score']} com={r['num_comments']} | {r['title'][:140]}")


[UTC window] 2025-09-24T00:00:00+00:00 -> 2025-09-25T00:00:00+00:00 (D=2025-09-24)

[collect] r/ethereum ...

[collect] r/ethfinance ...

[collect] r/ethtrader ...

[collect] r/CryptoCurrency ...

[collect] r/defi ...

[collect] r/ethdev ...

[summary]
  window (UTC): 2025-09-24T00:00:00+00:00 → 2025-09-25T00:00:00+00:00
  collected rows: 34  (after de-dup)
  time coverage:  2025-09-24 00:00:42+00:00 → 2025-09-24 23:23:18+00:00
  subreddits:     CryptoCurrency, defi, ethdev, ethereum, ethtrader
  saved csv:      ./data/reddit/yesterday\reddit_eth_submissions_2025-09-24.csv

[top 10 by score/num_comments]
- [2025-09-24T11:50:30+00:00] r/CryptoCurrency | score=168 com=44 | Over 30,000 BTC ($3.39B at $113K) were moved to exchanges at a loss as short-term holders capitulate
- [2025-09-24T05:00:40+00:00] r/ethereum | score=157 com=123 | Daily General Discussion September 24, 2025
- [2025-09-24T01:52:58+00:00] r/CryptoCurrency | score=70 com=4 | Morgan Stanley To Start Trading Bitcoin And Cr

In [3]:
# === Normalize yesterday's grabbed CSV to your legacy KEEP_FIELDS schema ===
# 输入:  data/reddit/yesterday/reddit_eth_submissions_YYYY-MM-DD.csv
# 输出:  data/reddit/processed/reddit_eth_standard_YYYY-MM-DD.csv
# 目的:  将新抓取的列映射/补全为旧逻辑使用的 17 个标准字段，方便后续沿用原有打分/聚合代码

import os
import pandas as pd
import datetime as dt

# 你的旧 KEEP_FIELDS（来自 sentiment.ipynb）
KEEP_FIELDS = [
    "id", "author", "subreddit",
    "created_utc", "created", "created_time_utc",
    "title", "selftext", "body",
    "url", "permalink",
    "score", "upvote_ratio", "num_comments", "num_crossposts",
    "over_18", "is_self"
]

# 昨天 UTC 的文件名（和抓取脚本一致）
D = (dt.datetime.now(dt.timezone.utc) - dt.timedelta(days=1)).date()
in_dir  = os.path.join("data", "reddit", "yesterday")
in_csv  = os.path.join(in_dir, f"reddit_eth_submissions_{D}.csv")

out_dir = os.path.join("data", "reddit", "processed")
os.makedirs(out_dir, exist_ok=True)
out_csv = os.path.join(out_dir, f"reddit_eth_standard_{D}.csv")

if not os.path.exists(in_csv):
    raise FileNotFoundError(f"找不到输入文件: {os.path.abspath(in_csv)}")

df = pd.read_csv(in_csv)

# ---- 字段映射/补全（最小改动，尽量复用已有列） ----
out = pd.DataFrame()

# 直接映射的列（若不存在则给空）
def pick(col, default=None):
    return df[col] if col in df.columns else default

out["id"]         = pick("id")
out["author"]     = pick("author")
out["subreddit"]  = pick("subreddit")
out["created_utc"]= pick("created_utc")  # 我们抓取时就是 ISO 字符串

# 由于你旧流程里既有 created_utc，也有 created/created_time_utc：
# 这里为兼容：created = 去掉时区的简洁显示；created_time_utc = 同 created_utc
ts = pd.to_datetime(out["created_utc"], errors="coerce", utc=True)
out["created"]          = ts.dt.tz_convert(None).dt.strftime("%Y-%m-%d %H:%M:%S")
out["created_time_utc"] = out["created_utc"]

out["title"]    = pick("title", "")
out["selftext"] = pick("selftext", "")
out["body"]     = ""  # 旧 schema 给评论字段，这里抓的是帖子，留空

out["url"]       = pick("url", "")
out["permalink"] = pick("permalink", "")

out["score"]        = pick("score", 0).fillna(0).astype("Int64")
out["upvote_ratio"] = pick("upvote_ratio", 0.0).fillna(0.0)
out["num_comments"] = pick("num_comments", 0).fillna(0).astype("Int64")

# num_crossposts：抓取里没有显式列；用 crosspost_parent 是否存在来近似
if "num_crossposts" in df.columns:
    out["num_crossposts"] = df["num_crossposts"].fillna(0).astype("Int64")
else:
    out["num_crossposts"] = df.get("crosspost_parent").notna().astype(int)

out["over_18"] = pick("over_18", False).fillna(False).astype(bool)
out["is_self"] = pick("is_self", False).fillna(False).astype(bool)

# 只保留旧 KEEP_FIELDS 的顺序与列名
out = out[KEEP_FIELDS]

# 去重（按 id）
out = out.sort_values(["id", "score"], ascending=[True, False]).drop_duplicates("id").reset_index(drop=True)

# 保存 CSV
out.to_csv(out_csv, index=False, encoding="utf-8-sig")

# 摘要
print("[normalize] 输入:", os.path.abspath(in_csv))
print("[normalize] 输出:", os.path.abspath(out_csv))
print("[normalize] 行数:", len(out))
print("[normalize] 列:", list(out.columns))
print(out[["created_utc","subreddit","score","num_comments","title"]].head(5))


[normalize] 输入: C:\Users\Jimmy\Desktop\760\data\reddit\yesterday\reddit_eth_submissions_2025-09-24.csv
[normalize] 输出: C:\Users\Jimmy\Desktop\760\data\reddit\processed\reddit_eth_standard_2025-09-24.csv
[normalize] 行数: 34
[normalize] 列: ['id', 'author', 'subreddit', 'created_utc', 'created', 'created_time_utc', 'title', 'selftext', 'body', 'url', 'permalink', 'score', 'upvote_ratio', 'num_comments', 'num_crossposts', 'over_18', 'is_self']
                 created_utc       subreddit  score  num_comments  \
0  2025-09-24T00:00:42+00:00       ethtrader      7            48   
1  2025-09-24T00:00:46+00:00  CryptoCurrency     22           390   
2  2025-09-24T01:52:58+00:00  CryptoCurrency     70             4   
3  2025-09-24T03:10:06+00:00  CryptoCurrency      5             2   
4  2025-09-24T04:37:08+00:00       ethtrader     59             7   

                                               title  
0  Daily General Discussion - September 24, 2025 ...  
1  Daily Crypto Discussion - Sep

In [4]:
# === Cell 1: 清洗 processed 文件 -> cleaned/reddit_eth_standard_{D}_clean.csv ===
import os, re, glob
import pandas as pd
import datetime as dt
from datetime import timezone

# ---- 自动定位输入文件 ----
D = (dt.datetime.now(timezone.utc) - dt.timedelta(days=1)).date()
default_path = f"data/reddit/processed/reddit_eth_standard_{D}.csv"
if os.path.exists(default_path):
    INPUT_CSV = default_path
else:
    # 兜底：找 processed 目录下最新的 reddit_eth_standard_*.csv
    cand = sorted(glob.glob("data/reddit/processed/reddit_eth_standard_*.csv"))
    if not cand:
        raise FileNotFoundError("找不到 processed 文件：data/reddit/processed/reddit_eth_standard_*.csv")
    INPUT_CSV = cand[-1]
    # 同步 D（从文件名里取日期）
    try:
        D = os.path.basename(INPUT_CSV).split("_")[-1].split(".")[0]
    except Exception:
        pass

OUT_DIR   = "cleaned"
MAX_LEN   = 20000
MIN_LEN   = 5
os.makedirs(OUT_DIR, exist_ok=True)

def clean_text(text: str) -> str:
    if pd.isna(text):
        return ""
    s = str(text)
    s = re.sub(r"\[([^\]]+)\]\(([^)]+)\)", r"\1", s)               # markdown link -> text
    s = re.sub(r"http\S+|www\.\S+", " ", s, flags=re.IGNORECASE)   # URLs
    s = re.sub(r"<[^>]+>", " ", s)                                 # HTML tags
    s = re.sub(r"[^A-Za-z0-9\s]", " ", s)                          # 只保留字母数字空格
    s = re.sub(r"\s+", " ", s).strip().lower()
    return s

# 读取
try:
    df = pd.read_csv(INPUT_CSV, low_memory=False)
except UnicodeDecodeError:
    df = pd.read_csv(INPUT_CSV, encoding="latin-1", low_memory=False)

# 统一文本列
if "title" not in df.columns:
    raise ValueError("CSV 缺少 'title' 列")
if "selftext" not in df.columns: df["selftext"] = ""
if "body" not in df.columns: df["body"] = ""

# 清理多余列
df = df.drop(columns=[c for c in df.columns if str(c).startswith("Unnamed")], errors="ignore")
df = df.dropna(subset=["title"])

# 拼接原始文本
df["text_raw"] = (df["title"].astype(str).fillna("") + " " + df["selftext"].astype(str).fillna(""))
mask_short = df["text_raw"].str.len().fillna(0) < 3
df.loc[mask_short, "text_raw"] = df.loc[mask_short, "text_raw"] + " " + df.loc[mask_short, "body"].astype(str)

# 截断 & 清洗
df["text_raw"] = df["text_raw"].astype(str).str.slice(0, MAX_LEN)
df["text_clean"] = df["text_raw"].map(clean_text)
df = df[df["text_clean"].str.len() >= MIN_LEN].copy()

# 解析时间（兼容 ISO 或 数字时间戳）
def parse_created_any(s):
    ts = pd.to_datetime(s, errors="coerce", utc=True)  # 先试 ISO
    if ts.isna().mean() > 0.5:  # 多数 NaT，尝试数字
        c = pd.to_numeric(s, errors="coerce")
        if c.notna().any():
            unit = "ms" if (c.dropna().median() > 10**12) else "s"
            ts = pd.to_datetime(c, unit=unit, errors="coerce", utc=True)
    return ts

if "created_time_utc" not in df.columns:
    if "created_utc" in df.columns:
        df["created_time_utc"] = parse_created_any(df["created_utc"])
    elif "created" in df.columns:
        df["created_time_utc"] = parse_created_any(df["created"])
    else:
        df["created_time_utc"] = pd.NaT

# year_month
if df["created_time_utc"].notna().any():
    df["year_month"] = pd.to_datetime(df["created_time_utc"]).dt.strftime("%Y-%m")
elif "source_file" in df.columns:
    df["year_month"] = df["source_file"].str.extract(r'((20\d{2})[-_](\d{2}))')[0]
else:
    df["year_month"] = None

# 去重
before = len(df)
if "id" in df.columns:
    df = df.sort_values(["id","score"] if "score" in df.columns else ["id"]) \
           .drop_duplicates("id")
else:
    keys = [k for k in ["title","created_time_utc","subreddit"] if k in df.columns]
    df = df.drop_duplicates(subset=keys) if keys else df.drop_duplicates()
after = len(df)

# 保存
base = os.path.splitext(os.path.basename(INPUT_CSV))[0]
out_path = os.path.join(OUT_DIR, f"{base}_clean.csv")
df.to_csv(out_path, index=False, encoding="utf-8-sig")

print("✅ 清洗完成")
print(" - 输入:", os.path.abspath(INPUT_CSV))
print(" - 输出:", os.path.abspath(out_path))
print(f" - 记录数: {after}（去重前 {before}）")
print(" - 关键列存在：", [c for c in ["text_raw","text_clean","created_time_utc","year_month"] if c in df.columns])
df.head(3)



✅ 清洗完成
 - 输入: C:\Users\Jimmy\Desktop\760\data\reddit\processed\reddit_eth_standard_2025-09-24.csv
 - 输出: C:\Users\Jimmy\Desktop\760\cleaned\reddit_eth_standard_2025-09-24_clean.csv
 - 记录数: 34（去重前 34）
 - 关键列存在： ['text_raw', 'text_clean', 'created_time_utc', 'year_month']


,id,author,subreddit,created_utc,created,created_time_utc,title,selftext,body,url,permalink,score,upvote_ratio,num_comments,num_crossposts,over_18,is_self,text_raw,text_clean,year_month
0,1noxjbk,AutoModerator,ethtrader,2025-09-24T00:00:42+00:00,2025-09-24 00:00:42,2025-09-24T00:00:42+00:00,"Daily General Discussion - September 24, 2025 ...",Welcome to the Daily General Discussion thread...,NaN,https://www.reddit.com/r/ethtrader/comments/1n...,https://www.reddit.com/r/ethtrader/comments/1n...,7,0.89,48,0,False,True,"Daily General Discussion - September 24, 2025 ...",daily general discussion september 24 2025 utc...,2025-09
1,1noxje4,AutoModerator,CryptoCurrency,2025-09-24T00:00:46+00:00,2025-09-24 00:00:46,2025-09-24T00:00:46+00:00,"Daily Crypto Discussion - September 24, 2025 (...",**Welcome to the Daily Crypto Discussion threa...,NaN,https://www.reddit.com/r/CryptoCurrency/commen...,https://www.reddit.com/r/CryptoCurrency/commen...,22,0.83,390,0,False,True,"Daily Crypto Discussion - September 24, 2025 (...",daily crypto discussion september 24 2025 gmt ...,2025-09
2,1nozvws,LavishlyRitzyy,CryptoCurrency,2025-09-24T01:52:58+00:00,2025-09-24 01:52:58,2025-09-24T01:52:58+00:00,Morgan Stanley To Start Trading Bitcoin And Cr...,NaN,NaN,https://blockchainreporter.net/morgan-stanley-...,https://www.reddit.com/r/CryptoCurrency/commen...,70,0.94,4,0,False,False,Morgan Stanley To Start Trading Bitcoin And Cr...,morgan stanley to start trading bitcoin and cr...,2025-09


In [5]:
# === Cell 2: 对 cleaned 文件做 VADER 打分，并写入主表 posts_scores_{D}.csv ===
import os, pandas as pd, datetime as dt
from datetime import timezone

# 输入（上一格输出）
D = (dt.datetime.now(timezone.utc) - dt.timedelta(days=1)).date()
clean_in = f"cleaned/reddit_eth_standard_{D}_clean.csv"
if not os.path.exists(clean_in):
    # 若你用的是“最新文件兜底”，名字可能不是昨天日期；找 cleaned 目录最新的 _clean.csv
    import glob
    cand = sorted(glob.glob("cleaned/*_clean.csv"))
    if not cand:
        raise FileNotFoundError("找不到 cleaned 文件")
    clean_in = cand[-1]

# 读取
df = pd.read_csv(clean_in)

# 选主表基列
base_cols = [c for c in [
    "id","subreddit","created_time_utc","title","selftext","body",
    "text_clean","score","num_comments","upvote_ratio","permalink","url"
] if c in df.columns]
dfb = df[base_cols].copy()

# VADER
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
try:
    _ = nltk.data.find("sentiment/vader_lexicon.zip")
except LookupError:
    nltk.download("vader_lexicon")

text_col = "text_clean" if "text_clean" in dfb.columns else None
if text_col is None:
    def _join(*xs): return " ".join([str(x) for x in xs if pd.notna(x)])
    dfb["__text_tmp__"] = [_join(t, s, b) for t, s, b in zip(
        dfb.get("title",""), dfb.get("selftext",""), dfb.get("body","")
    )]
    text_col = "__text_tmp__"

sia = SentimentIntensityAnalyzer()
dfb["vader"] = dfb[text_col].map(lambda t: sia.polarity_scores(str(t))["compound"])

# 主表保存（后续可继续在这个文件追加 s1…s5 列）
os.makedirs("data/reddit/scored", exist_ok=True)
master_p = f"data/reddit/scored/posts_scores_{D}.csv"
tmp_p = master_p + ".tmp"
dfb.to_csv(tmp_p, index=False, encoding="utf-8-sig")
os.replace(tmp_p, master_p)

print("✅ VADER 完成 & 主表已保存：", os.path.abspath(master_p))
print(" - 列：", list(dfb.columns))
print(" - 行：", len(dfb))
dfb[["id","vader","title"]].head(3)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Jimmy\AppData\Roaming\nltk_data...


✅ VADER 完成 & 主表已保存： C:\Users\Jimmy\Desktop\760\data\reddit\scored\posts_scores_2025-09-24.csv
 - 列： ['id', 'subreddit', 'created_time_utc', 'title', 'selftext', 'body', 'text_clean', 'score', 'num_comments', 'upvote_ratio', 'permalink', 'url', 'vader']
 - 行： 34


,id,vader,title
0,1noxjbk,0.9509,"Daily General Discussion - September 24, 2025 ..."
1,1noxje4,0.9801,"Daily Crypto Discussion - September 24, 2025 (..."
2,1nozvws,0.0000,Morgan Stanley To Start Trading Bitcoin And Cr...


In [6]:
# =========================
# Part 3: 本地 LM Studio 小模型打分 -> 在主表追加列 s1
# =========================
# 依赖：pip install openai pandas numpy tqdm nest_asyncio

import os, re, glob, time, asyncio
import numpy as np
import pandas as pd
import nest_asyncio
nest_asyncio.apply()
from tqdm.auto import tqdm
from openai import AsyncOpenAI
import datetime as dt
from datetime import timezone

# -------- LM Studio 基本配置 --------
LMSTUDIO_BASE_URL = "http://127.0.0.1:1234/v1"
LMSTUDIO_API_KEY  = "lm-studio"
MODEL_NAME        = "meta-llama-3.1-8b-instruct"   # LM Studio 右侧 Info 的 API identifier
OUT_COL           = "s1"                           # 本模型输出列名（你的第一个小模型）

# -------- 主表路径（含 vader 列）--------
D = (dt.datetime.now(timezone.utc) - dt.timedelta(days=1)).date()
INPUT_CSV  = f"data/reddit/scored/posts_scores_{D}.csv"  # 主表（上一步已写入 vader）
OUTPUT_CSV = INPUT_CSV                                   # 就地写回（原子写）

if not os.path.exists(INPUT_CSV):
    # 兜底：找 scored 目录下最新的 posts_scores_*.csv
    cands = sorted(glob.glob("data/reddit/scored/posts_scores_*.csv"), key=os.path.getmtime)
    if not cands:
        raise FileNotFoundError("找不到主表 posts_scores_{D}.csv，请先完成 VADER 步骤。")
    INPUT_CSV = OUTPUT_CSV = cands[-1]

TEXT_COL   = "text_clean"    # 优先用清洗文本
VADER_COL  = "vader"         # 可用于挑选“难例”再打分（可选）

# -------- 提速与控制参数 --------
MAX_TEXT_LEN = 256      # 截断避免上下文过长
VADER_EDGE   = None     # 仅对 |vader|<阈值 的样本打分；设为 None 则全量打分
CONCURRENCY  = 8        # 并发数
RETRY        = 3

# ——提示词（注意大括号需转义成双大括号）——
PROMPT = (
    "Classify the sentiment as exactly one token from {{POS, NEU, NEG}}.\n"
    "Text:\n{text}\n"
    "Answer:"
)
# 仅允许 POS/NEU/NEG（llama.cpp grammar）
GRAMMAR = r'root ::= "POS" | "NEU" | "NEG"'

def map_label_to_score(label: str) -> float:
    s = (label or "").strip().upper()
    first = re.split(r"\s+", s)[0] if s else ""
    if first in {"POS","NEU","NEG"}:
        return 1.0 if first=="POS" else (0.0 if first=="NEU" else -1.0)
    if "POS" in s: return 1.0
    if "NEG" in s: return -1.0
    if "NEU" in s: return 0.0
    return 0.0

# -------- 读取主表 & 选择待打样本 --------
try:
    df = pd.read_csv(INPUT_CSV, low_memory=False)
except UnicodeDecodeError:
    df = pd.read_csv(INPUT_CSV, encoding="latin-1", low_memory=False)

# 文本列兜底：若没有 text_clean，则用 title+selftext+body 合成
if TEXT_COL not in df.columns:
    def _join(*xs): return " ".join([str(x) for x in xs if pd.notna(x)])
    df["__text_tmp__"] = [_join(t, s, b) for t, s, b in zip(
        df.get("title",""), df.get("selftext",""), df.get("body","")
    )]
    TEXT_COL = "__text_tmp__"

# 确保待写列存在（新列先置 NaN）
if OUT_COL not in df.columns:
    df[OUT_COL] = np.nan

mask = df[OUT_COL].isna()
if VADER_EDGE is not None and VADER_COL in df.columns:
    mask &= df[VADER_COL].abs() < VADER_EDGE  # 仅给模糊样本补打分

todo = df[mask].copy()
if todo.empty:
    print(f"没有需要新打分的样本（{OUT_COL} 已存在或被阈值过滤）。文件：{INPUT_CSV}")
else:
    # 文本去重以减少请求
    todo["__txt"] = todo[TEXT_COL].astype(str).str.slice(0, MAX_TEXT_LEN)
    groups = todo.groupby("__txt").indices
    unique_texts = list(groups.keys())
    print(f"主表: {os.path.basename(INPUT_CSV)}")
    print(f"待打唯一文本数: {len(unique_texts)}（原样本 {len(todo)} / 全量 {len(df)}）")

    aclient = AsyncOpenAI(base_url=LMSTUDIO_BASE_URL, api_key=LMSTUDIO_API_KEY)

    async def classify_text(t: str) -> float:
        msg = PROMPT.format(text=t)
        for attempt in range(RETRY):
            try:
                resp = await aclient.chat.completions.create(
                    model=MODEL_NAME,
                    messages=[{"role":"user","content": msg}],
                    temperature=0,
                    max_tokens=1,
                    top_p=1,
                    stop=["\n"],
                    extra_body={"grammar": GRAMMAR}
                )
                return map_label_to_score(resp.choices[0].message.content)
            except Exception:
                if attempt == RETRY - 1:
                    return np.nan
                await asyncio.sleep(0.6 * (attempt + 1))

    async def run_all(texts, concurrency=8):
        sem = asyncio.Semaphore(concurrency)
        results = {}

        async def bound_task(t):
            async with sem:
                score = await classify_text(t)
            return t, score

        tasks = [asyncio.create_task(bound_task(t)) for t in texts]
        with tqdm(total=len(tasks), desc=f"{MODEL_NAME} → {OUT_COL}", unit="req") as pbar:
            for fut in asyncio.as_completed(tasks):
                t, sc = await fut
                results[t] = sc
                pbar.update(1)
        return results

    loop = asyncio.get_event_loop()
    scores_map = loop.run_until_complete(run_all(unique_texts, CONCURRENCY))

    # 回填 & 原子落盘
    for t, idxs in groups.items():
        df.loc[idxs, OUT_COL] = scores_map.get(t, np.nan)

    tmp_out = OUTPUT_CSV + ".tmp"
    df.to_csv(tmp_out, index=False, encoding="utf-8-sig")
    os.replace(tmp_out, OUTPUT_CSV)

    print(f"✅ 已写回列 {OUT_COL} -> {OUTPUT_CSV}")


主表: posts_scores_2025-09-24.csv
待打唯一文本数: 32（原样本 34 / 全量 34）


meta-llama-3.1-8b-instruct → s1:   0%|          | 0/32 [00:00<?, ?req/s]

✅ 已写回列 s1 -> data/reddit/scored/posts_scores_2025-09-24.csv


In [10]:
# === s2: Gemma-2-9B 在主表补齐列 s2（更稳健，带失败重试与兜底）===
import os, re, glob, asyncio, numpy as np, pandas as pd
import nest_asyncio; nest_asyncio.apply()
from tqdm.auto import tqdm
from openai import AsyncOpenAI
import datetime as dt
from datetime import timezone

LMSTUDIO_BASE_URL = "http://127.0.0.1:1234/v1"
LMSTUDIO_API_KEY  = "lm-studio"
MODEL_NAME        = "google/gemma-2-9b"
OUT_COL           = "s2"

# ——配置：全量打分、降低并发、加大重试——
VADER_EDGE = None          # 不过滤
DEDUP_UNIQUE_TEXTS = True  # 文本去重（失败会影响同文本多行；更快）
MAX_TEXT_LEN = 256
CONCURRENCY = 4            # ↓更稳（按需调高）
RETRY = 5                  # ↑更稳
FALLBACK_TO_VADER_SIGN = True  # 最终仍失败时，用 vader 的符号兜底

# ——路径——
D = (dt.datetime.now(timezone.utc) - dt.timedelta(days=1)).date()
INPUT_CSV = f"data/reddit/scored/posts_scores_{D}.csv"
if not os.path.exists(INPUT_CSV):
    cands = sorted(glob.glob("data/reddit/scored/posts_scores_*.csv"), key=os.path.getmtime)
    if not cands: raise FileNotFoundError("找不到主表 posts_scores_*.csv")
    INPUT_CSV = cands[-1]
OUTPUT_CSV = INPUT_CSV

df = pd.read_csv(INPUT_CSV, low_memory=False)

# 文本列兜底
TEXT_COL = "text_clean"
if TEXT_COL not in df.columns:
    def _join(*xs): return " ".join([str(x) for x in xs if pd.notna(x)])
    df["__text_tmp__"] = [_join(t, s, b) for t, s, b in zip(
        df.get("title",""), df.get("selftext",""), df.get("body","")
    )]
    TEXT_COL = "__text_tmp__"

# 准备要打分的行
if OUT_COL not in df.columns: df[OUT_COL] = np.nan
mask = df[OUT_COL].isna()
if (VADER_EDGE is not None) and ("vader" in df.columns):
    mask &= df["vader"].abs() < VADER_EDGE

todo = df.loc[mask].copy()
if todo.empty:
    print(f"没有需要打分的样本（{OUT_COL} 已存在或被阈值过滤）。文件：{INPUT_CSV}")
else:
    todo["__txt"] = todo[TEXT_COL].astype(str).str.slice(0, MAX_TEXT_LEN)

    if DEDUP_UNIQUE_TEXTS:
        groups = todo.groupby("__txt").indices
        work_items = list(groups.keys())  # 唯一文本
        def write_back(scores_map):
            for t, idxs in groups.items():
                df.loc[idxs, OUT_COL] = scores_map.get(t, np.nan)
    else:
        work_items = list(todo.index)     # 行索引
        def write_back(scores_map):
            for idx, sc in scores_map.items():
                df.loc[idx, OUT_COL] = sc

    print(f"主表: {os.path.basename(INPUT_CSV)} | 待打: {len(work_items)}（去重={DEDUP_UNIQUE_TEXTS}）")

    PROMPT = (
        "Classify the sentiment as exactly one token from {{POS, NEU, NEG}}.\n"
        "Text:\n{text}\n"
        "Answer:"
    )
    GRAMMAR = r'root ::= "POS" | "NEU" | "NEG"'

    def map_label_to_score(label: str) -> float:
        s = (label or "").strip().upper()
        first = re.split(r"\s+", s)[0] if s else ""
        if first in {"POS","NEU","NEG"}:
            return 1.0 if first=="POS" else (0.0 if first=="NEU" else -1.0)
        if "POS" in s: return 1.0
        if "NEG" in s: return -1.0
        if "NEU" in s: return 0.0
        return 0.0

    aclient = AsyncOpenAI(base_url=LMSTUDIO_BASE_URL, api_key=LMSTUDIO_API_KEY)

    async def ask(text) -> float:
        msg = PROMPT.format(text=text)
        for attempt in range(RETRY):
            try:
                r = await aclient.chat.completions.create(
                    model=MODEL_NAME,
                    messages=[{"role":"user","content": msg}],
                    temperature=0,
                    max_tokens=1,
                    top_p=1,
                    stop=["\n"],
                    extra_body={"grammar": GRAMMAR}
                )
                return map_label_to_score(r.choices[0].message.content)
            except Exception:
                # 按指数退避
                await asyncio.sleep(0.7 * (attempt + 1))
                if attempt == RETRY - 1:
                    return np.nan

    async def run_batch(items, desc):
        sem = asyncio.Semaphore(CONCURRENCY)
        results = {}
        async def one(item):
            text = item if DEDUP_UNIQUE_TEXTS else todo.at[item, "__txt"]
            async with sem:
                sc = await ask(text)
            return item, sc
        tasks = [asyncio.create_task(one(x)) for x in items]
        with tqdm(total=len(tasks), desc=desc, unit="req") as pbar:
            for fut in asyncio.as_completed(tasks):
                k, sc = await fut
                results[k] = sc
                pbar.update(1)
        return results

    loop = asyncio.get_event_loop()
    scores = loop.run_until_complete(run_batch(work_items, f"{MODEL_NAME} → {OUT_COL}"))

    # 二次只重打失败的（NaN 的）项
    failed = [k for k, v in scores.items() if (v is None) or (isinstance(v, float) and np.isnan(v))]
    if failed:
        print(f"⚠️ 首轮失败 {len(failed)} 条，降并发重试…")
        CONCURRENCY = max(2, CONCURRENCY // 2)
        retry_scores = loop.run_until_complete(run_batch(failed, f"retry {MODEL_NAME} → {OUT_COL}"))
        scores.update(retry_scores)

    # 写回
    write_back(scores)

    # 最终兜底：仍 NaN 的用 vader 符号或 0
    still_nan = df[OUT_COL].isna().sum()
    if still_nan and FALLBACK_TO_VADER_SIGN:
        print(f"⚠️ 仍有 {still_nan} 条 NaN，用 vader 符号兜底。")
        sign = np.sign(df.get("vader", 0.0).fillna(0.0))
        df.loc[df[OUT_COL].isna(), OUT_COL] = sign.replace(0, 0.0)

    tmp = OUTPUT_CSV + ".tmp"
    df.to_csv(tmp, index=False, encoding="utf-8-sig")
    os.replace(tmp, OUTPUT_CSV)
    print(f"✅ 已写回列 {OUT_COL} -> {OUTPUT_CSV}")


主表: posts_scores_2025-09-24.csv | 待打: 32（去重=True）


google/gemma-2-9b → s2:   0%|          | 0/32 [00:00<?, ?req/s]

✅ 已写回列 s2 -> data/reddit/scored/posts_scores_2025-09-24.csv


In [11]:
# =========================
# s3：相对路径读取/写回 data/reddit/scored/posts_scores_{D}.csv（全量打分）
# =========================
# pip install openai pandas numpy tqdm nest_asyncio

import os, re, glob, asyncio, numpy as np, pandas as pd
import nest_asyncio; nest_asyncio.apply()
from tqdm.auto import tqdm
from openai import AsyncOpenAI
import datetime as dt

# ---- 路径（相对项目根目录）----
D = (dt.datetime.utcnow().date() - dt.timedelta(days=1))
MASTER_DIR = "data/reddit/scored"
PREF = os.path.join(MASTER_DIR, f"posts_scores_{D}.csv")

if os.path.exists(PREF):
    MASTER_PATH = PREF
else:
    cands = sorted(glob.glob(os.path.join(MASTER_DIR, "posts_scores_*.csv")), key=os.path.getmtime)
    if not cands:
        raise FileNotFoundError("未找到主表：data/reddit/scored/posts_scores_*.csv，请先跑 VADER 初始化主表。")
    MASTER_PATH = cands[-1]

# ---- LM Studio 配置 ----
LMSTUDIO_BASE_URL = "http://127.0.0.1:1234/v1"
LMSTUDIO_API_KEY  = "lm-studio"
MODEL_NAME        = "qwen2.5-7b-instruct-1m"   # ← 改成 LM Studio 右侧的 API identifier
OUT_COL           = "s3"

# ---- 打分参数（全量）----
TEXT_COL     = "text_clean"
MAX_TEXT_LEN = 256
CONCURRENCY  = 8
RETRY        = 3

PROMPT = (
    "Classify the sentiment as exactly one token from {{POS, NEU, NEG}}.\n"
    "Text:\n{text}\n"
    "Answer:"
)
GRAMMAR = r'root ::= "POS" | "NEU" | "NEG"'

def map_label_to_score(label: str) -> float:
    s = (label or "").strip().upper()
    first = re.split(r"\s+", s)[0] if s else ""
    if first in {"POS","NEU","NEG"}:
        return 1.0 if first=="POS" else (0.0 if first=="NEU" else -1.0)
    if "POS" in s: return 1.0
    if "NEG" in s: return -1.0
    if "NEU" in s: return 0.0
    return 0.0

# ---- 读取主表 & 选择待打样本（全量，补缺）----
df = pd.read_csv(MASTER_PATH, low_memory=False)

if TEXT_COL not in df.columns:
    def _join(*xs): return " ".join([str(x) for x in xs if pd.notna(x)])
    df["__text_tmp__"] = [_join(t, s, b) for t, s, b in zip(
        df.get("title",""), df.get("selftext",""), df.get("body","")
    )]
    TEXT_COL = "__text_tmp__"

if OUT_COL not in df.columns:
    df[OUT_COL] = np.nan

mask = df[OUT_COL].isna()       # 只补缺
todo = df.loc[mask].copy()

if todo.empty:
    print(f"没有需要打分的样本（{OUT_COL} 已存在且无缺失）。文件：{MASTER_PATH}")
else:
    todo["__txt"] = todo[TEXT_COL].astype(str).str.slice(0, MAX_TEXT_LEN)
    groups = todo.groupby("__txt").indices   # 文本去重提速
    unique_texts = list(groups.keys())
    print(f"目标文件: {MASTER_PATH}")
    print(f"待打唯一文本数: {len(unique_texts)}（原样本 {len(todo)} / 全量 {len(df)}）")

    aclient = AsyncOpenAI(base_url=LMSTUDIO_BASE_URL, api_key=LMSTUDIO_API_KEY)

    async def classify_text(t: str) -> float:
        msg = PROMPT.format(text=t)
        for attempt in range(RETRY):
            try:
                r = await aclient.chat.completions.create(
                    model=MODEL_NAME,
                    messages=[{"role":"user","content": msg}],
                    temperature=0,
                    max_tokens=1,
                    top_p=1,
                    stop=["\n"],
                    extra_body={"grammar": GRAMMAR}
                )
                return map_label_to_score(r.choices[0].message.content)
            except Exception:
                if attempt == RETRY - 1:
                    return np.nan
                await asyncio.sleep(0.6 * (attempt + 1))

    async def run_all(texts, concurrency=8):
        sem = asyncio.Semaphore(concurrency)
        results = {}
        async def bound_task(t):
            async with sem:
                sc = await classify_text(t)
            return t, sc
        tasks = [asyncio.create_task(bound_task(t)) for t in texts]
        with tqdm(total=len(tasks), desc=f"{MODEL_NAME} → {OUT_COL}", unit="req") as pbar:
            for fut in asyncio.as_completed(tasks):
                t, sc = await fut
                results[t] = sc
                pbar.update(1)
        return results

    loop = asyncio.get_event_loop()
    scores_map = loop.run_until_complete(run_all(unique_texts, CONCURRENCY))

    # 回填 & 原子落盘（同一文件）
    for t, idxs in groups.items():
        df.loc[idxs, OUT_COL] = scores_map.get(t, np.nan)

    tmp_out = MASTER_PATH + ".tmp"
    df.to_csv(tmp_out, index=False, encoding="utf-8-sig")
    os.replace(tmp_out, MASTER_PATH)

    print(f"✅ 已写回列 {OUT_COL} -> {MASTER_PATH}")


C:\Users\Jimmy\AppData\Local\Temp\ipykernel_15604\3966319938.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  D = (dt.datetime.utcnow().date() - dt.timedelta(days=1))


目标文件: data/reddit/scored\posts_scores_2025-09-24.csv
待打唯一文本数: 32（原样本 34 / 全量 34）


qwen2.5-7b-instruct-1m → s3:   0%|          | 0/32 [00:00<?, ?req/s]

✅ 已写回列 s3 -> data/reddit/scored\posts_scores_2025-09-24.csv


In [12]:
# =========================
# s4：LM Studio 本地小模型打分（全量补缺）→ 追加列 s4
# =========================
# 依赖：pip install openai pandas numpy tqdm nest_asyncio

import os, re, glob, asyncio, numpy as np, pandas as pd
import nest_asyncio; nest_asyncio.apply()
from tqdm.auto import tqdm
from openai import AsyncOpenAI
import datetime as dt

# ---- 路径（相对项目根）----
D = (dt.datetime.utcnow().date() - dt.timedelta(days=1))
MASTER_DIR  = "data/reddit/scored"
PREF        = os.path.join(MASTER_DIR, f"posts_scores_{D}.csv")
MASTER_PATH = PREF if os.path.exists(PREF) else sorted(
    glob.glob(os.path.join(MASTER_DIR, "posts_scores_*.csv")),
    key=os.path.getmtime
)[-1]

# ---- LM Studio 配置（改这里的 MODEL_NAME）----
LMSTUDIO_BASE_URL = "http://127.0.0.1:1234/v1"
LMSTUDIO_API_KEY  = "lm-studio"
MODEL_NAME        = "mistralai/mistral-7b-instruct-v0.3"   # ← 改成 LM Studio 右侧 API identifier
OUT_COL           = "s4"

# ---- 打分参数 ----
TEXT_COL     = "text_clean"
MAX_TEXT_LEN = 256
CONCURRENCY  = 8
RETRY        = 3
DEDUP_UNIQUE_TEXTS = True   # True：同文只打一次（快）；False：逐行必打

PROMPT = (
    "Classify the sentiment as exactly one token from {{POS, NEU, NEG}}.\n"
    "Text:\n{text}\n"
    "Answer:"
)
GRAMMAR = r'root ::= "POS" | "NEU" | "NEG"'

def map_label_to_score(label: str) -> float:
    s = (label or "").strip().upper()
    first = re.split(r"\s+", s)[0] if s else ""
    if first in {"POS","NEU","NEG"}:
        return 1.0 if first=="POS" else (0.0 if first=="NEU" else -1.0)
    if "POS" in s: return 1.0
    if "NEG" in s: return -1.0
    if "NEU" in s: return 0.0
    return 0.0

# ---- 读取主表 & 选取待打样本（全量补缺）----
df = pd.read_csv(MASTER_PATH, low_memory=False)

if TEXT_COL not in df.columns:
    def _join(*xs): return " ".join([str(x) for x in xs if pd.notna(x)])
    df["__text_tmp__"] = [_join(t, s, b) for t, s, b in zip(
        df.get("title",""), df.get("selftext",""), df.get("body","")
    )]
    TEXT_COL = "__text_tmp__"

if OUT_COL not in df.columns:
    df[OUT_COL] = np.nan

todo = df.loc[df[OUT_COL].isna()].copy()
if todo.empty:
    print(f"没有需要打分的样本（{OUT_COL} 已存在且无缺失）。文件：{MASTER_PATH}")
else:
    if DEDUP_UNIQUE_TEXTS:
        todo["__txt"] = todo[TEXT_COL].astype(str).str.slice(0, MAX_TEXT_LEN)
        groups = todo.groupby("__txt").indices
        work_items = list(groups.keys())          # 唯一文本集合
        def write_back(scores_map):
            for t, idxs in groups.items():
                df.loc[idxs, OUT_COL] = scores_map.get(t, np.nan)
    else:
        todo["__txt"] = todo[TEXT_COL].astype(str).str.slice(0, MAX_TEXT_LEN)
        work_items = list(todo.index)             # 行索引集合
        def write_back(scores_map):
            for idx, sc in scores_map.items():
                df.loc[idx, OUT_COL] = sc

    print(f"目标文件: {MASTER_PATH}")
    print(f"准备打分: {len(work_items)}（去重={DEDUP_UNIQUE_TEXTS}）")

    aclient = AsyncOpenAI(base_url=LMSTUDIO_BASE_URL, api_key=LMSTUDIO_API_KEY)

    async def classify_text(text: str) -> float:
        msg = PROMPT.format(text=text)
        for attempt in range(RETRY):
            try:
                r = await aclient.chat.completions.create(
                    model=MODEL_NAME,
                    messages=[{"role":"user","content": msg}],
                    temperature=0,
                    max_tokens=1,
                    top_p=1,
                    stop=["\n"],
                    extra_body={"grammar": GRAMMAR}
                )
                return map_label_to_score(r.choices[0].message.content)
            except Exception:
                if attempt == RETRY - 1:
                    return np.nan
                await asyncio.sleep(0.6 * (attempt + 1))

    async def run_all(items):
        sem = asyncio.Semaphore(CONCURRENCY)
        results = {}
        async def run_one(item):
            text = item if DEDUP_UNIQUE_TEXTS else todo.at[item, "__txt"]
            async with sem:
                sc = await classify_text(text)
            return item, sc
        tasks = [asyncio.create_task(run_one(x)) for x in items]
        with tqdm(total=len(tasks), desc=f"{MODEL_NAME} → {OUT_COL}", unit="req") as pbar:
            for fut in asyncio.as_completed(tasks):
                k, sc = await fut
                results[k] = sc
                pbar.update(1)
        return results

    loop = asyncio.get_event_loop()
    scores_map = loop.run_until_complete(run_all(work_items))

    # 回填 & 原子落盘
    write_back(scores_map)
    tmp_out = MASTER_PATH + ".tmp"
    df.to_csv(tmp_out, index=False, encoding="utf-8-sig")
    os.replace(tmp_out, MASTER_PATH)

    print(f"✅ 已写回列 {OUT_COL} -> {MASTER_PATH}")


C:\Users\Jimmy\AppData\Local\Temp\ipykernel_15604\1081153019.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  D = (dt.datetime.utcnow().date() - dt.timedelta(days=1))


目标文件: data/reddit/scored\posts_scores_2025-09-24.csv
准备打分: 32（去重=True）


mistralai/mistral-7b-instruct-v0.3 → s4:   0%|          | 0/32 [00:00<?, ?req/s]

✅ 已写回列 s4 -> data/reddit/scored\posts_scores_2025-09-24.csv


In [13]:
# =========================
# s5：LM Studio 本地小模型打分（全量补缺）→ 追加列 s5
# =========================
# 依赖：pip install openai pandas numpy tqdm nest_asyncio

import os, re, glob, asyncio, numpy as np, pandas as pd
import nest_asyncio; nest_asyncio.apply()
from tqdm.auto import tqdm
from openai import AsyncOpenAI
import datetime as dt

# ---- 路径（相对项目根）----
D = (dt.datetime.utcnow().date() - dt.timedelta(days=1))
MASTER_DIR  = "data/reddit/scored"
PREF        = os.path.join(MASTER_DIR, f"posts_scores_{D}.csv")
MASTER_PATH = PREF if os.path.exists(PREF) else sorted(
    glob.glob(os.path.join(MASTER_DIR, "posts_scores_*.csv")),
    key=os.path.getmtime
)[-1]

# ---- LM Studio 配置（改这里的 MODEL_NAME）----
LMSTUDIO_BASE_URL = "http://127.0.0.1:1234/v1"
LMSTUDIO_API_KEY  = "lm-studio"
MODEL_NAME        = "nous-hermes-2-mistral-7b-dpo"   # ← 改成 LM Studio 右侧 API identifier
OUT_COL           = "s5"

# ---- 打分参数 ----
TEXT_COL     = "text_clean"
MAX_TEXT_LEN = 256
CONCURRENCY  = 8
RETRY        = 3
DEDUP_UNIQUE_TEXTS = True  # True: 同文只打一次；False: 逐行必打

PROMPT = (
    "Classify the sentiment as exactly one token from {{POS, NEU, NEG}}.\n"
    "Text:\n{text}\n"
    "Answer:"
)
GRAMMAR = r'root ::= "POS" | "NEU" | "NEG"'

def map_label_to_score(label: str) -> float:
    s = (label or "").strip().upper()
    first = re.split(r"\s+", s)[0] if s else ""
    if first in {"POS","NEU","NEG"}:
        return 1.0 if first=="POS" else (0.0 if first=="NEU" else -1.0)
    if "POS" in s: return 1.0
    if "NEG" in s: return -1.0
    if "NEU" in s: return 0.0
    return 0.0

# ---- 读取主表 & 选择待打样本（全量补缺）----
df = pd.read_csv(MASTER_PATH, low_memory=False)

if TEXT_COL not in df.columns:
    def _join(*xs): return " ".join([str(x) for x in xs if pd.notna(x)])
    df["__text_tmp__"] = [_join(t, s, b) for t, s, b in zip(
        df.get("title",""), df.get("selftext",""), df.get("body","")
    )]
    TEXT_COL = "__text_tmp__"

if OUT_COL not in df.columns:
    df[OUT_COL] = np.nan

todo = df.loc[df[OUT_COL].isna()].copy()
if todo.empty:
    print(f"没有需要打分的样本（{OUT_COL} 已存在且无缺失）。文件：{MASTER_PATH}")
else:
    if DEDUP_UNIQUE_TEXTS:
        todo["__txt"] = todo[TEXT_COL].astype(str).str.slice(0, MAX_TEXT_LEN)
        groups = todo.groupby("__txt").indices
        work_items = list(groups.keys())
        def write_back(scores_map):
            for t, idxs in groups.items():
                df.loc[idxs, OUT_COL] = scores_map.get(t, np.nan)
    else:
        todo["__txt"] = todo[TEXT_COL].astype(str).str.slice(0, MAX_TEXT_LEN)
        work_items = list(todo.index)
        def write_back(scores_map):
            for idx, sc in scores_map.items():
                df.loc[idx, OUT_COL] = sc

    print(f"目标文件: {MASTER_PATH}")
    print(f"准备打分: {len(work_items)}（去重={DEDUP_UNIQUE_TEXTS}）")

    aclient = AsyncOpenAI(base_url=LMSTUDIO_BASE_URL, api_key=LMSTUDIO_API_KEY)

    async def classify_text(text: str) -> float:
        msg = PROMPT.format(text=text)
        for attempt in range(RETRY):
            try:
                r = await aclient.chat.completions.create(
                    model=MODEL_NAME,
                    messages=[{"role":"user","content": msg}],
                    temperature=0,
                    max_tokens=1,
                    top_p=1,
                    stop=["\n"],
                    extra_body={"grammar": GRAMMAR}
                )
                return map_label_to_score(r.choices[0].message.content)
            except Exception:
                if attempt == RETRY - 1:
                    return np.nan
                await asyncio.sleep(0.6 * (attempt + 1))

    async def run_all(items):
        sem = asyncio.Semaphore(CONCURRENCY)
        results = {}
        async def run_one(item):
            text = item if DEDUP_UNIQUE_TEXTS else todo.at[item, "__txt"]
            async with sem:
                sc = await classify_text(text)
            return item, sc
        tasks = [asyncio.create_task(run_one(x)) for x in items]
        with tqdm(total=len(tasks), desc=f"{MODEL_NAME} → {OUT_COL}", unit="req") as pbar:
            for fut in asyncio.as_completed(tasks):
                k, sc = await fut
                results[k] = sc
                pbar.update(1)
        return results

    loop = asyncio.get_event_loop()
    scores_map = loop.run_until_complete(run_all(work_items))

    # 回填 & 原子落盘
    write_back(scores_map)
    tmp_out = MASTER_PATH + ".tmp"
    df.to_csv(tmp_out, index=False, encoding="utf-8-sig")
    os.replace(tmp_out, MASTER_PATH)

    print(f"✅ 已写回列 {OUT_COL} -> {MASTER_PATH}")


C:\Users\Jimmy\AppData\Local\Temp\ipykernel_15604\2155423926.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  D = (dt.datetime.utcnow().date() - dt.timedelta(days=1))


目标文件: data/reddit/scored\posts_scores_2025-09-24.csv
准备打分: 32（去重=True）


nous-hermes-2-mistral-7b-dpo → s5:   0%|          | 0/32 [00:00<?, ?req/s]

✅ 已写回列 s5 -> data/reddit/scored\posts_scores_2025-09-24.csv


In [15]:
# =========================
# Weighted daily -> 1min forward-fill (vader + s1..s5)
# 从 data/reddit/scored 聚合到分钟级；每天定值，展开成 1min
# =========================
# pip install pandas numpy

import os, glob, math
import pandas as pd
import numpy as np

# ---- 路径 ----
SCORED_DIR = "data/reddit/scored"
OUT_DIR    = "data/reddit/weighted"
os.makedirs(OUT_DIR, exist_ok=True)

# ---- 读取所有 posts_scores_*.csv ----
paths = sorted(glob.glob(os.path.join(SCORED_DIR, "posts_scores_*.csv")))
if not paths:
    raise FileNotFoundError("data/reddit/scored/ 下未找到 posts_scores_*.csv，请先生成主表。")

dfs = []
for p in paths:
    try:
        dfp = pd.read_csv(p, low_memory=False)
    except UnicodeDecodeError:
        dfp = pd.read_csv(p, encoding="latin-1", low_memory=False)
    dfp["__source"] = os.path.basename(p)
    dfs.append(dfp)

df = pd.concat(dfs, ignore_index=True)

# ---- 情绪列：兼容两种命名（s1..s5 或 sent_s1..sent_s5），统一成 s1..s5 ----
sent_cols = []
if "vader" in df.columns:
    sent_cols.append("vader")

# 优先 s1..s5；若不存在则映射 sent_s1..sent_s5 -> s1..s5
for k in ["s1","s2","s3","s4","s5"]:
    if k in df.columns:
        sent_cols.append(k)
    elif f"sent_{k}" in df.columns:
        df[k] = pd.to_numeric(df[f"sent_{k}"], errors="coerce")
        sent_cols.append(k)

if not sent_cols:
    raise ValueError("没有找到情绪列（期望 vader + s1..s5 中至少一列）。")

# ---- 时间列（优先 created_time_utc，其次 created_utc 秒）----
if "created_time_utc" in df.columns:
    t = pd.to_datetime(df["created_time_utc"], errors="coerce", utc=True)
elif "created_utc" in df.columns:
    t = pd.to_datetime(df["created_utc"], unit="s", errors="coerce", utc=True)
else:
    raise ValueError("需要 'created_time_utc' 或 'created_utc' 列。")

df["created_time_utc"] = t
df = df.dropna(subset=["created_time_utc"])
df = df[df["created_time_utc"] >= "2005-01-01"]

# ---- 去重（若有 id）----
if "id" in df.columns:
    df = df.sort_values("created_time_utc").drop_duplicates("id", keep="last")

# ---- 权重：log(1+score) + 0.5*log(1+num_comments) ----
score = pd.to_numeric(df.get("score", 0), errors="coerce").fillna(0).clip(lower=0)
if "num_comments" in df.columns:
    numc = pd.to_numeric(df["num_comments"], errors="coerce").fillna(0).clip(lower=0)
    weight = score.apply(math.log1p) + 0.5 * numc.apply(math.log1p)
else:
    weight = score.apply(math.log1p)
df["__w"] = weight

# ---- 情绪列转数值并裁剪到 [-1,1] ----
for c in sent_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce").clip(-1, 1)

# ---- 日聚合（UTC 天）：加权平均；若当日总权重=0，则退化为简单均值 ----
df["date_utc"] = df["created_time_utc"].dt.date

def wavg(series: pd.Series, w: pd.Series) -> float:
    s = series.astype(float)
    w = w.astype(float)
    den = np.nansum(w)
    return float(np.nansum(s * w) / den) if den > 0 else float(np.nanmean(s))

daily_rows = []
for d, g in df.groupby("date_utc"):
    row = {"date_utc": d}
    for c in sent_cols:
        row[c] = wavg(g[c], g["__w"])
    daily_rows.append(row)

daily = pd.DataFrame(daily_rows).sort_values("date_utc").reset_index(drop=True)

# ---- 扩展到每分钟并前向填充（merge_asof，更稳）----
# 1) 日度时间戳设为当天 00:00 UTC
daily_ff = daily.copy()
daily_ff["ts_day"] = pd.to_datetime(daily_ff["date_utc"]).dt.tz_localize("UTC")
daily_ff = daily_ff.sort_values("ts_day").reset_index(drop=True)

# 2) 生成分钟索引（UTC）
start = daily_ff["ts_day"].min()
end   = daily_ff["ts_day"].max() + pd.Timedelta(days=1) - pd.Timedelta(minutes=1)
minute_df = pd.DataFrame({"ts": pd.date_range(start=start, end=end, freq="min", tz="UTC")})

# 3) asof 向后合并：每分钟取“最近且不晚于它的日度 00:00”值
joined = pd.merge_asof(
    minute_df.sort_values("ts"),
    daily_ff[["ts_day"] + sent_cols].sort_values("ts_day"),
    left_on="ts", right_on="ts_day",
    direction="backward"
)

# 4) 清理列，只保留 ts + 情绪列
minute_df = joined.drop(columns=["ts_day"])
minute_df = minute_df[["ts"] + sent_cols]

# ---- 保存 ----
daily_out  = os.path.join(OUT_DIR, "sentiment_daily_vader_s1_s5.csv")
minute_out = os.path.join(OUT_DIR, "sentiment_1min_vader_s1_s5.csv")

daily_out_df = daily.rename(columns={"date_utc": "ts"})
daily_out_df["ts"] = pd.to_datetime(daily_out_df["ts"]).dt.tz_localize("UTC")

daily_out_df.to_csv(daily_out, index=False, encoding="utf-8")
minute_df.to_csv(minute_out, index=False, encoding="utf-8")

print("✅ Done.")
print(f"Days   : {daily_out_df['ts'].min().date()} ~ {daily_out_df['ts'].max().date()}")
print(f"Daily  : {daily_out}")
print(f"1-min  : {minute_out}")
print("\nPreview (daily):")
display(daily_out_df.head(3))
print("\nPreview (1-min):")
display(minute_df.head(3))



✅ Done.
Days   : 2025-09-24 ~ 2025-09-24
Daily  : data/reddit/weighted\sentiment_daily_vader_s1_s5.csv
1-min  : data/reddit/weighted\sentiment_1min_vader_s1_s5.csv

Preview (daily):


,ts,vader,s1,s2,s3,s4,s5
0,2025-09-24 00:00:00+00:00,0.4048,-0.23979,0.035424,-0.152483,-0.232451,-0.167206



Preview (1-min):


,ts,vader,s1,s2,s3,s4,s5
0,2025-09-24 00:00:00+00:00,0.4048,-0.23979,0.035424,-0.152483,-0.232451,-0.167206
1,2025-09-24 00:01:00+00:00,0.4048,-0.23979,0.035424,-0.152483,-0.232451,-0.167206
2,2025-09-24 00:02:00+00:00,0.4048,-0.23979,0.035424,-0.152483,-0.232451,-0.167206
